<h2 align="center"> Data Mining and Machine Learning </h2>
<h3 align="center"> Final Project </h3>
<h2 align="center"> <b> <i> CrashSpot </i> </b> </h2>
<h4 align="center"> Lorenzo Ceccanti matr. 564490 </h4>

### <b> Geografical Clustering </b>

We start considering the dataset in which the granularity is per accident in order to perform a geographical clustering.

First of all, we check the Hopkins statistic in order to measure how far is our dataset to be uniformly distributed. 

If the Hopkins index stays as low as possible and far from 1 (as suggests pyclustertend's documentation it should be higher than 0.3) we have a clustering tendency.

In [94]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyclustertend import hopkins
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

R_EARTH_KM = 6371.0

df = pd.read_csv(os.path.join('editedDataset', 'CLEANED_brasilEnglishAggr.csv'))

In [95]:
# The theory suggests us to sample uniformly the dataset picking up a number of points lower lower than the total number of instances
# we have. We do so by using the sample method on the DataFrame df contained our cleaned dataset

no_samples = 5000

X_lat_lon = df[['latitude', 'longitude']]
# We pick only latitude and longitude attributes as they are the one on which the clustering activity focuses in the first phase
X = df[['latitude', 'longitude']].sample(5000, random_state=42)
X_elbow = X
small_n = X.shape[0]
# The conversion is necessary since hopkins() expects an ndarray as first parameter
H = hopkins(X.to_numpy(), small_n)
H

0.015664442154403938

Trying to normalize the coordinates using the MinMaxScaler only before looking at the hopkins statistics. 
This because according to some implementations of hopkins(), the distance may be computed using the Euclidean distance.

In [96]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# The theory suggests us to sample uniformly the dataset picking up a number of points lower lower than the total number of instances
# we have. We do so by using the sample method on the DataFrame df contained our cleaned dataset

no_samples = 5000
scaler = MinMaxScaler()

# We pick only latitude and longitude attributes as they are the one on which the clustering activity focuses in the first phase
X = df[['latitude', 'longitude']]
X_scaled = scaler.fit_transform(X)

N = X_scaled.shape[0]

# Sampling with replacement, with ndarray. Equivalent of sample for pandas df
np.random.seed(42)
indexes = np.random.choice(N, size = 5000, replace = None)
X_sampled = X_scaled[indexes,:]

small_n = X_sampled.shape[0]
# The conversion is necessary since hopkins() expects an ndarray as first parameter
H = hopkins(X_sampled, small_n)
H

0.01634356689621129

Since in both the cases, we obtained a value for H very very near to 0, we can conclude that we have a clustering tendency.

At this point, we decide to perform a geographical clustering with the DBSCAN algorithm as suggested the reference papers.

Before starting to apply the DBSCAN, it's important to look at how to choose the required parameters.

In particular, we use the Elbow method to select approprietaly a value for `Eps` and we assume `k=MinPts` as we saw during the theoretical lectures.

The following function help us to construct a k-dist graph.

In [97]:
import plotly.express as px
def plot_k_dist_graph(X,k):

    # Converting in radians
    coords_rad = np.radians(X[['latitude', 'longitude']].to_numpy())

    # For each object, we consider its distance with the k-th nearest neighbor
    neigh = NearestNeighbors(n_neighbors=k, metric="haversine", algorithm="ball_tree")
    neigh.fit(coords_rad)
    # The kneighbors method of neighb object returns the distance as the first
    # parameter and the index of the nearest point as second parameter
    # However, we are not interested in the second parameter
    distances_rad, _ = neigh.kneighbors(coords_rad)
    # the k-1 column in distances is the distance with the k-th nearest neighbor
    # : is to consider the distance between EACH point and that k-th neighbor
    # We consider a decreasing order, that is not implemented in numpy

    kth_rad = distances_rad[:,k-1]
    kth_km = kth_rad * R_EARTH_KM

    kth_km_sorted = np.sort(kth_km)[::-1]
    
    distances_df = pd.DataFrame({
        # Here I'm creating the indexes for the x-axis (the number of points)
        "Points": range(len(kth_km_sorted)),
        f"{k}-th nearest neighbor distance": kth_km_sorted
    })
    fig = px.line(distances_df, x = 'Points', y = f'{k}-th nearest neighbor distance', title = 'K-distance Graph')
    fig.show()

In [98]:
MinPts = 4
distances = plot_k_dist_graph(X_elbow, MinPts)

In [99]:
MinPts = 2
distances = plot_k_dist_graph(X_elbow, MinPts)

### DBSCAN: reasoning in radiants

In [117]:
X_elbow.head(3)

,latitude,longitude
308389,-19.78306,-45.25944
267817,-12.77929,-60.33542
305393,-16.48424,-49.09291


In [118]:
X_elbow
coords = np.radians(X_elbow[['latitude', 'longitude']])
coords.head(3)

,latitude,longitude
308389,-0.345280,-0.789926
267817,-0.223041,-1.053052
305393,-0.287704,-0.856833


### DBSCAN: Grid search for parameter tuning

In [ ]:
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
def get_GeoDBSCAN_metrics(eps_km, minPts, data):
    # Fit
    dbscan = DBSCAN(eps = eps_km/R_EARTH_KM, min_samples=minPts, metric='haversine')
    dbscan.fit(data)

    labels = dbscan.labels_
    # Preparing a mask to exclude the indexes belonging to outliers
    mask = labels != -1

    # Here we have only the data which are not considered as outliers by the DBSCAN
    data_filtered = data[mask]
    labels_filtered = labels[mask]

    # First metric: Percentage of core points and number of outliers
    count_core_points = np.sum(mask)
    total_points = np.size(mask)
    count_outliers = total_points - count_core_points

    # Second metric: Number of clusters
    number_of_clusters = len(set(dbscan.labels_[dbscan.labels_ >= 0]))

    # Third metric: Davies-Bouldin Index
    davies_bouldin_index = davies_bouldin_score(data_filtered, labels_filtered)

    # Fourth metric: Silouette Coefficent
    sil_coefficent = metrics.silhouette_score(data_filtered, labels_filtered, metric='haversine')

    # Fifth metrics: Calinski-Harabasz index
    calinski_index = metrics.calinski_harabasz_score(data_filtered, labels_filtered)

    # I return a dictionary containing the metrics associated with the Eps and MinPts chosen
    df_results = {
        'eps_km': eps_km,
        'min_samples': minPts,
        'count_core_points': count_core_points,
        'count_outliers': count_outliers,
        'number_of_clusters': number_of_clusters,
        'davies_bouldin_index': davies_bouldin_index,
        'sil_coefficent': sil_coefficent,
        'calinski_index': calinski_index
    }
    return df_results, labels    

In [ ]:
def dbscan_tuning(minEps, maxEps, stepEps, minPtsArr, data):
    
    eps_to_test = []
    for eps in np.arange(minEps, maxEps, stepEps):
        eps_to_test.append(eps)
    
    print(f'Eps to test: {eps_to_test}')
    print(f'MinPts to test: {minPtsArr}')

    # It's an array of dictionaries
    arr_tuning_results = []

    for eps in eps_to_test:
        for minPts in minPtsArr:
            df_metric, _ = get_GeoDBSCAN_metrics(eps, minPts, data)
            arr_tuning_results.append(df_metric)

    df_tuning_results = pd.DataFrame(arr_tuning_results)
    return df_tuning_results

In [ ]:
minEps = 5
maxEps = 10
stepEps = 0.25
minPtsArr = [2,4,6,8,10]
tuning_results = dbscan_tuning(minEps, maxEps, stepEps, minPtsArr, coords)
tuning_results

In [115]:
tuning_results = tuning_results.sort_values(
    by=["davies_bouldin_index", "sil_coefficent", "calinski_index"],
    ascending=[True, False, False]
)
tuning_results

,eps_km,min_samples,count_core_points,count_outliers,number_of_clusters,davies_bouldin_index,sil_coefficent,calinski_index
39,6.75,10,1940,3060,69,0.217051,0.782600,367010.676219
34,6.50,10,1905,3095,70,0.217307,0.779823,392076.720730
33,6.50,8,2166,2834,88,0.223763,0.766899,323219.790812
44,7.00,10,1986,3014,70,0.224096,0.776083,355077.567385
28,6.25,8,2113,2887,85,0.226428,0.759125,333851.331756
...,...,...,...,...,...,...,...,...
96,9.75,4,3764,1236,230,0.311994,0.527485,24372.840137
91,9.50,4,3740,1260,231,0.312062,0.526331,24497.047052
86,9.25,4,3721,1279,235,0.315737,0.539489,31738.983399
81,9.00,4,3678,1322,236,0.316814,0.535119,32149.180072


### Graphical Visualization of the Clusters Obtained

In [ ]:
_, labels = get_GeoDBSCAN_metrics(eps_km=6.75, minPts=10, data=coords)

In [ ]:
# Forcing to return a copy of the X_elbow object, not its reference
X_elbow_labeled = X_elbow.copy()
X_elbow_labeled['label'] = labels

In [42]:
X_elbow_labeled.loc[:,'label']

308389     -1
267817     -1
305393     -1
122584      0
101183    106
         ... 
76175      11
162707    251
436013     -1
332602     -1
288288     -1
Name: label, Length: 5000, dtype: int64

In [43]:
import plotly.express as px

color_scale = [
    [0.0,  "white"],
    [0.1,  "lightgray"],
    [0.2,  "lightblue"],
    [0.3,  "blue"],
    [0.4,  "green"],
    [0.5,  "lightgreen"],
    [0.6,  "yellow"],
    [0.7,  "gold"],
    [0.8,  "orange"],
    [0.9,  "orangered"],
    [1.0,  "red"]
]

fig = px.scatter_mapbox(
    X_elbow_labeled,
    lat="latitude",
    lon="longitude",
    color="label",       # continuo, non stringa
    zoom=3,
    height=600,
    opacity=0.8,
    color_continuous_scale=color_scale
)

fig.update_layout(
    mapbox_style="carto-positron",
    title="Cluster su mappa geografica"
)

fig.show()